In [897]:
import numpy as np
import os  
import time

# this functios is for bipolarization, mapping '.' to -1 and '#' to 1 and for test data turning 'O' to -1 and '@' to 1
# getting a list of string and bipolarizing the data
def bipolarization(listString):
    
    l = list(listString)
    
    for i in range(len(l)):

        if l[i] == '.':
            l[i] = -1
            
        if l[i] == '#':
            l[i] = 1
            
        if l[i] == '@':
            l[i] = 1
            
        if l[i] == 'O':
            l[i] = -1

    return l

# this function is used for encoding characters 
def targetEncoding(string):
    
    if string == "A":
        return np.array([1,-1,-1,-1,-1,-1,-1])
    if string == "B":
        return np.array([-1,1,-1,-1,-1,-1,-1])
    if string == "C":
        return np.array([-1,-1,1,-1,-1,-1,-1])
    if string == "D":
        return np.array([-1,-1,-1,1,-1,-1,-1])
    if string == "E":
        return np.array([-1,-1,-1,-1,1,-1,-1])
    if string == "K":
        return np.array([-1,-1,-1,-1,-1,1,-1])
    if string == "J":
        return np.array([-1,-1,-1,-1,-1,-1,1])
    
def readingData(dataSetName):
    
    # getting each files name for further encoding
    allFiles = os.listdir(dataSetName)   # we are in one directory above Characters-TrainSet dir
    
    # data size refers to the number of raws in our dataset
    dataSize = len(allFiles)

    defaultPath = "/Users/kiarash/Desktop/SoftComputing/Characters-TrainSet/"

    #here we are defined our input data as "X" as our target as "Y"
    X = []
    Y = []

    for i in range(dataSize):

        path = defaultPath + allFiles[i]
        
        # encoding each target list to a 7*1 vector
        encodedTarget = targetEncoding(allFiles[i][0])
        
        # creating target array
        Y.append(encodedTarget)
        
        # deleting \n characters
        f = open(path)  
        txtData = f.read()
        data = txtData.replace("\n", "")
        
        # bipolarizing input and saving as X
        bipolarInput = bipolarization(data)
        X.append(bipolarInput)

        path = defaultPath
        
    Y = np.array(Y)
    X = np.array(X)
    
    return X, Y, dataSize

dataSetName = "Characters-TrainSet"

XTrain,YTrain, dataSize = readingData(dataSetName)

print("X's Shape is:" + str(X.shape),"and Y's Shape is:" + str(Y.shape))

dataSetName = "Characters-TestSet"

XTest,YTest, dataSize = readingData(dataSetName)

XTrain, XTest

X's Shape is:(21, 63) and Y's Shape is:(21, 7)


(array([[ 1,  1,  1, ..., -1,  1,  1],
        [ 1, -1, -1, ..., -1,  1, -1],
        [ 1,  1,  1, ..., -1,  1,  1],
        ...,
        [ 1,  1,  1, ...,  1, -1, -1],
        [ 1,  1,  1, ...,  1, -1, -1],
        [ 1,  1,  1, ...,  1, -1, -1]]),
 array([[ 1,  1,  1, ..., -1,  1,  1],
        [ 1, -1, -1, ..., -1,  1, -1],
        [ 1,  1,  1, ..., -1,  1,  1],
        ...,
        [ 1,  1,  1, ...,  1, -1, -1],
        [ 1,  1,  1, ...,  1, -1, -1],
        [ 1,  1,  1, ...,  1, -1, -1]]))

In [898]:
class Model:
    
    
    def __init__(self, x, target, weight, bias, alpha, theta, dataSize):
        self.x = x
        self.weight = weight
        self.bias = bias
        self.alpha = alpha
        self.theta = theta
        self.target = target
        self.dataSize = dataSize
        
    def ActivationFunction(self, i):
        
        if((i > -self.theta) and (i < self.theta)):
            return 0
        elif(i > self.theta):
            return 1
        elif(i < -self.theta):
            return -1        
        
    # this fucntion is used for perceptron learning algorithm  
    def perceptronLearn(self):
        
        y = np.empty((21,7))

        tic = time.time()
        accList = []
        epoch = 0
        tempt = 0
        
        while(True):
            
            epoch +=  1    
            
            for i in range(self.dataSize) :  
                for j in range (7) :
                    
                    yIn = self.bias[0][j] + (self.x[i,:] @ self.weight[:,j])
                    yPrime = self.ActivationFunction(yIn)

                    if(yPrime != self.target[i][j]):
                        
                        deltaWeight = self.alpha * (self.x[i,:] * self.target[i][j])
                        self.weight[:,j] += deltaWeight
                        self.bias[0][j] += self.alpha * self.target[i][j]
                        
                        y[i][j] = yPrime
                        
                    else:
                        y[i][j] = yPrime
                        
            # finding accuracy
            for i in range(self.dataSize):
                temp = (y[i] == self.target[i])
                
                if temp.all() == True :
                    tempt += 1
                    
            acc = ( tempt / self.dataSize) * 100
            accList.append(acc)
                
            if (acc > 96 or epoch > 100):
                # here our learning algorithm ends, there for self.weight and self.bias represent
                # the best possible weight and bias
                self.t = time.time() - tic
                
                self.bestW = self.weight
                self.bestB = self.bias
                
                return accList, epoch
            tempt = 0
            
    # this fucntion is used for Adaline learning algorithm  
    def adalineLearn(self):
        
        y = np.empty((21,7))
    
        tic = time.time()
        accList = []
        epoch = 0
        tempt = 0
        
        while(True):     
            
            epoch +=  1    
            
            for i in range(self.dataSize) :  
                for j in range (7) :
                    
                    yIn = self.bias[0][j] + (self.x[i,:] @ self.weight[:,j])
                    yPrime = self.ActivationFunction(yIn)
                    
                    if(yPrime != self.target[i][j]):
                        
                        deltaWeight = self.alpha * (self.x[i,:] * (self.target[i][j] - yIn))
                        self.weight[:,j] += deltaWeight
                        self.bias[0][j] += self.alpha * (self.target[i][j] - yIn)

                        y[i][j] = yPrime
                        
                    else:
                        y[i][j] = yPrime
            
            # finding accuracy
            for i in range(self.dataSize):
                temp = (y[i] == self.target[i])
                
                if temp.all() != True :
                    tempt += 1

            acc = (1 - (tempt / self.dataSize)) * 100
            accList.append(acc)
            
            # here our learning algorithm ends, there for self.weight and self.bias represent
            # the best possible weight and bias    
            if (acc > 96 or epoch > 100):
                
                self.t = time.time() - tic
                
                self.bestW = self.weight
                self.bestB = self.bias
                
                return accList, epoch
            
            tempt = 0
            
    # this function is for prediction based on our final weights        
    def predict(self, x, target):
        
        yHat = np.empty((self.dataSize,7)) 
        tempt = 0
        
        for i in range(self.dataSize):  
            for j in range (7):  
                    
                yIn = self.bestB[0][j] + (x[i,:] @ self.bestW[:,j])
                yPrime = self.ActivationFunction(yIn)
                    
                yHat[i][j] = yPrime
            
        # finding accuracy
        for i in range(self.dataSize):
            temp = (yHat[i] == target[i])
                
            if temp.all() == True :
                tempt += 1

        acc = ((tempt / self.dataSize) ) * 100
        return acc        

In [899]:
rand = np.random.RandomState(42)
weight0 = randW.random((63,7))
bias = randB.random((1,7))

M = Model(XTrain, YTrain, weight0, bias, 0.1, 0, dataSize)
trainAccuracy, epoch =  M.perceptronLearn()

trainAccuracy, epoch

([14.285714285714285,
  47.61904761904761,
  85.71428571428571,
  80.95238095238095,
  100.0],
 5)

In [900]:
rand = np.random.RandomState(42)
weight0 = randW.random((63,7))
bias = randB.random((1,7))

M = Model(XTrain, YTrain, weight0, bias, 0.1, 0, dataSize)
trainAccuracy, epoch =  M.adalineLearn()

trainAccuracy, epoch

([47.61904761904761,
  52.38095238095239,
  57.14285714285714,
  71.42857142857143,
  57.14285714285714,
  95.23809523809523,
  100.0],
 7)

In [902]:
# 2.1 a

for seedI in [40,42,44]:
    for seedJ in [40,42,44]:

        randW = np.random.RandomState(seedI)
        randB = np.random.RandomState(seedJ)

        weight0 = randW.random((63,7))
        bias = randB.random((1,7))

        M = Model(XTrain, YTrain, weight0, bias, 0.2, 0.2, dataSize)
        trainAccuracy, epoch =  M.perceptronLearn()
        
        testAccuracy = M.predict(XTest, YTest)
        
        print('Accuracy on train for (weightSeed,biasSeed)=' + '(' + str(seedI) +','+ str(seedJ) + ')' + ' is: ' 
            + str(round(trainAccuracy[-1],4)) + ' and takes ' + str(round(M.t,4)) + ' time and accuracy on test is: ' 
                + str(round(testAccuracy,4)))
        print()

        # best seed for random initialized weight is equal to 42 and Bias equal to 44

Accuracy on train for (weightSeed,biasSeed)=(40,40) is: 100.0 and takes 0.0142 time and accuracy on test is: 100.0

Accuracy on train for (weightSeed,biasSeed)=(40,42) is: 100.0 and takes 0.0074 time and accuracy on test is: 100.0

Accuracy on train for (weightSeed,biasSeed)=(40,44) is: 100.0 and takes 0.0083 time and accuracy on test is: 100.0

Accuracy on train for (weightSeed,biasSeed)=(42,40) is: 100.0 and takes 0.007 time and accuracy on test is: 100.0

Accuracy on train for (weightSeed,biasSeed)=(42,42) is: 100.0 and takes 0.0063 time and accuracy on test is: 100.0

Accuracy on train for (weightSeed,biasSeed)=(42,44) is: 100.0 and takes 0.0058 time and accuracy on test is: 100.0

Accuracy on train for (weightSeed,biasSeed)=(44,40) is: 100.0 and takes 0.0045 time and accuracy on test is: 100.0

Accuracy on train for (weightSeed,biasSeed)=(44,42) is: 100.0 and takes 0.0038 time and accuracy on test is: 100.0

Accuracy on train for (weightSeed,biasSeed)=(44,44) is: 100.0 and takes 0

In [903]:
# 2.1 b

for theta in [0, 0.01, 0.05, 0.2, 0.5, 0.99]:
    
    rand = np.random.RandomState(42)
    weight0 = rand.random((63,7))

    rand = np.random.RandomState(44)
    bias = rand.random((1,7))

    M = Model(XTrain, YTrain, weight0, bias, 0.2, theta, dataSize)
    accuracy, epoch =  M.perceptronLearn()
    
    testAccuracy = M.predict(XTest, YTest)
    
    print('Best accuracy on test data theta ' + str(theta) + ' is: ' + str(testAccuracy))
    

Best accuracy on test data theta 0 is: 100.0
Best accuracy on test data theta 0.01 is: 100.0
Best accuracy on test data theta 0.05 is: 100.0
Best accuracy on test data theta 0.2 is: 100.0
Best accuracy on test data theta 0.5 is: 100.0
Best accuracy on test data theta 0.99 is: 100.0


In [904]:
# 2.1 c 

for alpha in [0.02, 0.1, 0.3, 0.5]:
    
    rand = np.random.RandomState(42)
    weight0 = rand.random((63,7))

    rand = np.random.RandomState(44)
    bias = rand.random((1,7))

    M = Model(XTrain, YTrain, weight0, bias, alpha, 0.2, dataSize)
    accuracy, epoch =  M.perceptronLearn()
    testAccuracy = M.predict(XTest, YTest)

    print('Best accuracy for alpha ' + str(alpha) + ' is: ' + str(testAccuracy) + 
         ' and this algorithm takes ' + str(M.t) + ' time')

Best accuracy for alpha 0.02 is: 100.0 and this algorithm takes 0.027122020721435547 time
Best accuracy for alpha 0.1 is: 100.0 and this algorithm takes 0.008373022079467773 time
Best accuracy for alpha 0.3 is: 100.0 and this algorithm takes 0.007465362548828125 time
Best accuracy for alpha 0.5 is: 100.0 and this algorithm takes 0.005498170852661133 time


In [905]:
rand = np.random.RandomState(42)
weight0 = rand.random((63,7))

rand = np.random.RandomState(44)
bias = rand.random((1,7))

for alpha in [0.02, 0.1, 0.3, 0.5]:
    for theta in [0, 0.01, 0.5]:
        
        rand = np.random.RandomState(42)
        weight0 = rand.random((63,7))

        rand = np.random.RandomState(44)
        bias = rand.random((1,7))

        
        M = Model(XTrain, YTrain, weight0, bias, alpha, theta, dataSize)
        accuracy, epoch =  M.adalineLearn()
        testAccuracy = M.predict(XTest, YTest)
        
        print('Best accuracy for Adaline for (alpha,theta)=' + '(' + str(alpha) + ',' + str(theta) + ')'
              + ' on train and test is: ' + str(round(accuracy[-1],4)) + ',' + str(round(testAccuracy,4)) +
              ' training takes ' + str(round(M.t,4)) + ' time')

Best accuracy for Adaline for (alpha,theta)=(0.02,0) on train and test is: 100.0,100.0 training takes 0.0127 time
Best accuracy for Adaline for (alpha,theta)=(0.02,0.01) on train and test is: 100.0,100.0 training takes 0.0075 time
Best accuracy for Adaline for (alpha,theta)=(0.02,0.5) on train and test is: 100.0,100.0 training takes 0.0074 time
Best accuracy for Adaline for (alpha,theta)=(0.1,0) on train and test is: 100.0,100.0 training takes 0.0104 time
Best accuracy for Adaline for (alpha,theta)=(0.1,0.01) on train and test is: 100.0,100.0 training takes 0.0083 time
Best accuracy for Adaline for (alpha,theta)=(0.1,0.5) on train and test is: 100.0,100.0 training takes 0.0082 time
Best accuracy for Adaline for (alpha,theta)=(0.3,0) on train and test is: 52.381,14.2857 training takes 0.0715 time
Best accuracy for Adaline for (alpha,theta)=(0.3,0.01) on train and test is: 52.381,14.2857 training takes 0.0639 time
Best accuracy for Adaline for (alpha,theta)=(0.3,0.5) on train and test is

In [906]:
# feature extraction function that outputs the summation of column and raws in a 2d array
# notice that we are not normalizing data as the question does not see this necessary
# even though normalizing usually is highly suggested

def featureExtraction(array):
    
    #sum of columns
    res = [ (x > 0).sum() for x in array.T ]
    
    #sum of raws
    for x in array:
        res.append((x > 0).sum())
    
    return res


a = np.array([[-1,-2,3,-1], 
              [1,-2,3,-1],
              [-1,2,3,1]])

featureExtraction(a)

[1, 1, 3, 1, 1, 2, 3]

In [907]:
# in this section we need to reshape our data in order to use feature extraction

XPrimeTrain = []

for i in XTrain:
    
    # reshaping each data raw to a 9*7 matrix
    
    d = i.reshape(-1,7)
                
    raw = featureExtraction(d)
    
    XPrimeTrain.append(raw)
        
XPrimeTrain = np.array(XPrimeTrain)

XPrimeTrain

array([[2, 9, 4, 3, 2, 3, 2, 6, 2, 2, 2, 2, 2, 2, 2, 5],
       [9, 1, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2],
       [2, 9, 3, 2, 2, 4, 2, 5, 2, 2, 2, 2, 2, 2, 2, 5],
       [0, 1, 1, 1, 2, 8, 1, 3, 1, 1, 1, 1, 1, 1, 2, 3],
       [0, 2, 1, 1, 1, 8, 0, 1, 1, 1, 1, 1, 1, 2, 2, 3],
       [0, 2, 1, 2, 2, 8, 1, 4, 1, 1, 1, 1, 1, 2, 2, 3],
       [1, 4, 5, 4, 4, 4, 1, 2, 1, 1, 2, 2, 5, 2, 2, 6],
       [0, 4, 3, 4, 3, 4, 0, 1, 1, 1, 2, 2, 2, 5, 2, 2],
       [3, 3, 3, 3, 3, 3, 3, 1, 1, 2, 2, 2, 5, 2, 2, 4],
       [0, 7, 2, 2, 2, 2, 3, 5, 2, 1, 1, 1, 1, 1, 2, 4],
       [5, 2, 2, 2, 2, 2, 4, 4, 3, 2, 1, 1, 1, 2, 2, 3],
       [5, 2, 2, 2, 2, 2, 2, 3, 2, 2, 1, 1, 1, 2, 2, 3],
       [9, 3, 3, 3, 3, 3, 6, 6, 2, 2, 2, 6, 2, 2, 2, 6],
       [2, 9, 3, 3, 3, 3, 6, 6, 2, 2, 5, 2, 2, 2, 2, 6],
       [2, 9, 3, 3, 3, 3, 6, 6, 2, 2, 2, 5, 2, 2, 2, 6],
       [2, 9, 3, 3, 5, 2, 4, 7, 2, 2, 4, 2, 1, 1, 2, 7],
       [9, 3, 3, 3, 3, 2, 2, 7, 1, 1, 1, 5, 1, 1, 1, 7],
       [2, 9, 3, 5, 2, 2, 4, 7,

In [908]:
rand = np.random.RandomState(42)
weight0 = randW.random((16,7))
bias = randB.random((1,7))

M = Model(XPrimeTrain, YTrain, weight0, bias, 0.01, 0.5, dataSize)

trainAccuracy, epoch =  M.adalineLearn()

trainAccuracy, epoch, M.t

([47.61904761904761,
  47.61904761904761,
  42.85714285714286,
  38.095238095238095,
  23.809523809523814,
  28.57142857142857,
  23.809523809523814,
  38.095238095238095,
  33.333333333333336,
  38.095238095238095,
  52.38095238095239,
  52.38095238095239,
  57.14285714285714,
  52.38095238095239,
  42.85714285714286,
  61.904761904761905,
  61.904761904761905,
  52.38095238095239,
  66.66666666666667,
  57.14285714285714,
  66.66666666666667,
  71.42857142857143,
  90.47619047619048,
  90.47619047619048,
  90.47619047619048,
  90.47619047619048,
  95.23809523809523,
  100.0],
 28,
 0.0506129264831543)

In [909]:
rand = np.random.RandomState(42)
weight0 = randW.random((63,7))
bias = randB.random((1,7))

M = Model(XTrain, YTrain, weight0, bias, 0.1, 0.5, dataSize)

trainAccuracy, epoch =  M.adalineLearn()

trainAccuracy, epoch, M.t

([47.61904761904761,
  42.85714285714286,
  42.85714285714286,
  57.14285714285714,
  52.38095238095239,
  90.47619047619048,
  100.0],
 7,
 0.018394947052001953)

In [910]:
rand = np.random.RandomState(42)
weight0 = randW.random((63,7))
bias = randB.random((1,7))

M = Model(XTrain, YTrain, weight0, bias, 0.1, 0.5, dataSize)

trainAccuracy, epoch =  M.adalineLearn()

testAccuracy = M.predict(XTest, YTest)
trainAccuracy, epoch,testAccuracy

([52.38095238095239,
  66.66666666666667,
  42.85714285714286,
  61.904761904761905,
  80.95238095238095,
  76.19047619047619,
  90.47619047619048,
  90.47619047619048,
  100.0],
 9,
 100.0)